<a href="https://colab.research.google.com/github/EduardoMoraesRitter/NLP-Chatbot/blob/main/conversationAnalyticsToolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Note, on Watson Studio the pip magic command `%pip` is not supported from within the notebook.  Use !pip instead. 
#!pip install --user conversation_analytics_toolkit
%pip install --user conversation_analytics_toolkit

In [2]:
import nltk
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import conversation_analytics_toolkit

from conversation_analytics_toolkit import wa_assistant_skills, transformation, analysis, visualization, wa_adaptor, transcript, flows, keyword_analysis, sentiment_analysis
from conversation_analytics_toolkit import selection as vis_selection
from conversation_analytics_toolkit import filtering2 as filtering

import json
import pandas as pd
from pandas.io.json import json_normalize
from IPython.core.display import display, HTML

import ibm_watson
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# fill it out with your WAS credentials
workspace_id = ""
WAS_API_KEY = ""
WAS_URL = "https://api.us-east.assistant.watson.cloud.ibm.com"

In [ ]:
# set pandas to show more rows and columns
#pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)

Fetch and load a workspace

In [ ]:
authenticator = IAMAuthenticator(WAS_API_KEY)
service = AssistantV1(version='2019-02-20', authenticator = authenticator)

service.set_service_url(WAS_URL)

# or fetch one via the APIs
workspaces = service.list_workspaces().get_result()
#workspaces['workspaces']
#workspace_id = service['workspaces'][0]['workspace_id']
#workspace_id

In [ ]:
#fetch the workspace
workspace=service.get_workspace(workspace_id=workspace_id, export=True).get_result()
#workspace = assistant.get_workspace(workspace_id=check_workspace_id).get_result()
workspace

In [ ]:
# set query parameters
limit_number_of_records=2000

# example of time range query
query_filter = "response_timestamp>=2022-10-13,response_timestamp<2022-10-14" #query_filter = None

# Fetch the logs for the workspace
df_logs = wa_adaptor.read_logs(service, workspace_id, limit_number_of_records, query_filter)
#response = assistant.list_logs( workspace_id = wid, page_limit = pl ).get_result()
df_logs

Extract and Transform

In [ ]:
#if you have more than one skill, you can add multiple skill definitions
skill_id = workspace_id
assistant_skills = wa_assistant_skills.WA_Assistant_Skills()
assistant_skills.add_skill(skill_id, workspace)

#validate the number of workspace_ids
print("workspace_ids in skills: " + pd.DataFrame(assistant_skills.list_skills())["skill_id"].unique())
print("workspace_ids in logs: "+ df_logs.workspace_id.unique())

In [ ]:
df_logs_canonical = transformation.to_canonical_WA_v2(df_logs, assistant_skills, skill_id_field=None, include_nodes_visited_str_types=True, include_context=True);
#df_logs_canonical = transformation.to_canonical_WA_v2(df_logs, assistant_skills, skill_id_field="workspace_id", include_nodes_visited_str_types=True, include_context=False)
df_logs_canonical.head()

In [ ]:
# the rest of the notebook runs on the df_logs_to_analyze object.  
df_logs_to_analyze = df_logs_canonical.copy(deep=False)
df_logs_to_analyze.head(2)

Visualizing user journeys and abandonments

In [ ]:
title = "All Conversations"
turn_based_path_flows = analysis.aggregate_flows(df_logs_to_analyze, mode="turn-based", on_column="turn_label", max_depth=400, trim_reroutes=False)
# increase the width of the Jupyter output cell   
display(HTML("<style>.container { width:95% !important; }</style>"))
config = {
    'commonRootPathName': title, # label for the first root node 
    'height': 800, # control the visualization height.  Default 600
    'nodeWidth': 250, 
    'maxChildrenInNode': 6, # control the number of immediate children to show (and collapse rest into *others* node).  Default 5
    'linkWidth' : 400,  # control the width between pathflow layers.  Default 360     'sortByAttribute': 'flowRatio'  # control the sorting of the chart. (Options: flowRatio, dropped_offRatio, flows, dropped_off, rerouted)
    'sortByAttribute': 'flowRatio',
    'title': title,
    'mode': "turn-based"
}
jsondata = json.loads(turn_based_path_flows.to_json(orient='records'))
visualization.draw_flowchart(config, jsondata, python_selection_var="selection")

Visualize flows in all conversations

In [ ]:
title = "All Conversations"
turn_based_path_flows = analysis.aggregate_flows(df_logs_to_analyze, mode="turn-based", on_column="turn_label", max_depth=400, trim_reroutes=False)
# increase the width of the Jupyter output cell   
display(HTML("<style>.container { width:95% !important; }</style>"))
config = {
    'commonRootPathName': title, # label for the first root node 
    'height': 800, # control the visualization height.  Default 600
    'nodeWidth': 250, 
    'maxChildrenInNode': 6, # control the number of immediate children to show (and collapse rest into *others* node).  Default 5
    'linkWidth' : 400,  # control the width between pathflow layers.  Default 360     'sortByAttribute': 'flowRatio'  # control the sorting of the chart. (Options: flowRatio, dropped_offRatio, flows, dropped_off, rerouted)
    'sortByAttribute': 'flowRatio',
    'title': title,
    'mode': "turn-based"
}
jsondata = json.loads(turn_based_path_flows.to_json(orient='records'))
visualization.draw_flowchart(config, jsondata, python_selection_var="selection")

Visualize flows in subset of conversations

In [ ]:
# filter the conversations that include escalation
title2="Banking Card Escalated"
filters = filtering.ChainFilter(df_logs_to_analyze).setDescription(title2) 
# node with condition on the #Banking-Card_Selection (node_1_1510880732839) and visit the node "Transfer To Live Agent" (node_25_1516679473977)
#filters.by_dialog_node_id('node_1_1510880732839').by_dialog_node_id('node_25_1516679473977')
filters.printConversationFilters() 
# get a reference to the dataframe.  Note: you can get access to intermediate dataframes by calling getDataFrame(index)

In [ ]:
filtered_df = filters.getDataFrame()
turn_based_path_flows = analysis.aggregate_flows(filtered_df, mode="turn-based", on_column="turn_label", max_depth=400, trim_reroutes=False)  
config = {
    'commonRootPathName': title2,  'title': title2,
    'height': 800,  'nodeWidth': 250, 'maxChildrenInNode': 6, 'linkWidth' : 400, 'sortByAttribute': 'flowRatio',
    'mode': "turn-based"
}
jsondata = json.loads(turn_based_path_flows.to_json(orient='records'))
visualization.draw_flowchart(config, jsondata, python_selection_var="selection")

Analyzing reverse flows

In [ ]:
# filter only conversations that include Confirming Live Agent Transfer.  In addition, remove true nodes, which simplifies the journey
#removeing the true node, helps to make the visualization simpler
#cf = filtering.ChainFilter(df_logs_canonical).setDescription("Escalated conversations").by_turn_label("Confirming Live Agent Transfer").remove_turn_by_label("true").printConversationFilters()
cf = filtering.ChainFilter(df_logs_canonical).remove_turn_by_label("true").printConversationFilters()

escalated_df = cf.getDataFrame()
#addiginal remove duplicates consecutive states will also make the visualization simpler
escalated_simplified_df = analysis.simplify_consecutive_duplicates(escalated_df, on_column="turn_label")

In [ ]:
#note reverse=True in the aggregation step
escalated_similified_reversed_df = analysis.aggregate_flows(escalated_simplified_df, mode="turn-based", on_column="turn_label", max_depth=30, trim_reroutes=False, reverse=True)

In [ ]:
title3 =  'Escalation Flows (reversed)'
config = {
    'commonRootPathName': title3,  'title': title3,
    'height': 800,  'nodeWidth': 250, 'maxChildrenInNode': 6, 'linkWidth' : 400, 'sortByAttribute': 'flowRatio',
    'mode': "turn-based"
}
jsondata = json.loads(escalated_similified_reversed_df.to_json(orient='records'))
visualization.draw_flowchart(config, jsondata, python_selection_var="selection")

Analyzing trends in flows

In [ ]:
# we will simiulate this on the previous chart (reverse escalated flows) by comparing the first and second 1000 logs
prev_period_df = escalated_simplified_df[0:1000]
curr_period_df = escalated_simplified_df[1001:2000]

prev_flows = analysis.aggregate_flows(prev_period_df, mode="turn-based", on_column="turn_label", max_depth=30, trim_reroutes=False, reverse=True)
curr_flows = analysis.aggregate_flows(curr_period_df, mode="turn-based", on_column="turn_label", max_depth=30, trim_reroutes=False, reverse=True)

df_logs_compare_periods = analysis.merge_compare_flows(curr_flows, prev_flows)

In [ ]:
title3 =  'Trending flows (curr vs. prev)'
config = {
    'commonRootPathName': title3,  'title': title3,
    'height': 800,  'nodeWidth': 250, 'maxChildrenInNode': 6, 'linkWidth' : 400, 'sortByAttribute': 'flowRatio',
    'mode': "turn-based"
}
jsondata = json.loads(df_logs_compare_periods.to_json(orient='records'))
visualization.draw_flowchart(config, jsondata, python_selection_var="selection")

Visualize dialog flow (milestone-based)

In [ ]:
#define the milestones and corresponding node ids for the `Schedule Appointment` task
milestone_analysis = analysis.MilestoneFlowGraph(assistant_skills.get_skill_by_id(skill_id))
milestone_analysis
# milestone_analysis.add_milestones(["Appointment scheduling start",  "Schedule time", "Enter zip code", "Branch selection", 
#                     "Enter purpose of appointment", "Scheduling completion"])

# milestone_analysis.add_node_to_milestone("node_21_1513047983871", "Appointment scheduling start")   
# milestone_analysis.add_node_to_milestone("handler_28_1513048122602", "Schedule time")

In [ ]:
#enrich with milestone information - will add a column called 'milestone'
milestone_analysis.enrich_milestones(df_logs_to_analyze)
#remove all log records without a milestone
#df_milestones = df_logs_to_analyze[pd.isna(df_logs_to_analyze["milestone"]) == False]
df_milestones = df_logs_to_analyze[pd.isna(df_logs_to_analyze) == False]
#optionally, remove consecutive milestones for a more simplified flow visualization representation
df_milestones = analysis.simplify_flow_consecutive_milestones(df_milestones)

In [ ]:
# compute the aggregate flows of milestones 
computed_flows= analysis.aggregate_flows(df_milestones, mode="milestone-based", on_column="milestone", max_depth=30, trim_reroutes=False)
config = {
    'commonRootPathName': 'All Conversations', # label for the first root node 
    'height': 800, # control the visualization height.  Default 600
    'maxChildrenInNode': 6, # control the number of immediate children to show (and collapse the rest into *other* node).  Default 5
#     'linkWidth' : 400,  # control the width between pathflow layers.  Default 360     '
    'sortByAttribute': 'flowRatio', # control the sorting of the chart. (Options: flowRatio, dropped_offRatio, flows, dropped_off, rerouted)
    'title': "Abandoned Conversations in Appointment Schedule Flow",
    'showVisitRatio' : 'fromTotal', # default: 'fromTotal'.  'fromPrevious' will compute percentages from previous step,
    'mode': 'milestone-based'
}
jsondata = json.loads(computed_flows.to_json(orient='records'))
visualization.draw_flowchart(config, jsondata, python_selection_var="milestone_selection")

Select conversations at the point of abandonment

In [ ]:
#the selection variable contains details about the selected node, and conversations that were abandoned at that point
print("Selected Path: ",milestone_selection["path"])
#fetch the dropped off conversations from the selection  
dropped_off_conversations = vis_selection.to_dataframe(milestone_selection)["dropped_off"]
print("The selection contains {} records, with a reference back to the converstion logs".format(str(len(dropped_off_conversations))))
dropped_off_conversations.head()

Analyzing abandoned conversations

In [ ]:
df_logs_to_analyze = sentiment_analysis.add_sentiment_columns(df_logs_to_analyze) 
#create insights, and highlights annotation for the transcript visualization
NEGATIVE_SENTIMENT_THRESHOLD=-0.15 
df_logs_to_analyze["insights_tags"] = df_logs_to_analyze.apply(lambda x: ["Negative Sentiment"] if x.sentiment < NEGATIVE_SENTIMENT_THRESHOLD else [], axis=1)
df_logs_to_analyze["highlight"] = df_logs_to_analyze.apply(lambda x: True if x.sentiment < NEGATIVE_SENTIMENT_THRESHOLD else False, axis=1)

In [ ]:
# fetch the conversation records
dropped_off_conversations = vis_selection.fetch_logs_by_selection(df_logs_to_analyze, dropped_off_conversations)
# visualize using the transcript visualization 
dfc = transcript.to_transcript(dropped_off_conversations)
config = {'debugger': True} 
visualization.draw_transcript(config, dfc)

Identify key words and phrases at point of abandonment

In [ ]:
# gather user utterances from the dropped off conversations - last utterances and all utterances
last_utterances_abandoned=vis_selection.get_last_utterances_from_selection(milestone_selection, df_logs_to_analyze)
all_utterances_abandoned=vis_selection.get_all_utterances_from_selection(milestone_selection, df_logs_to_analyze)

Summarize frequent keywords and phrases

In [ ]:
# analyze the last user input before abandonment
num_unigrams=10
num_bigrams=15
custom_stop_words=["would","pm","ok","yes","no","thank","thanks","hi","i","you"]
data = keyword_analysis.get_frequent_words_bigrams(last_utterances_abandoned, num_unigrams,num_bigrams,custom_stop_words)    

In [ ]:
config = {'flattened': True, 'width' : 800, 'height' : 500}
visualization.draw_wordpackchart(config, data)